# Tutorial IV: Convolutions in pytorch

<p>
Bern Winter School on Machine Learning, 2024<br>
Prepared by Mykhailo Vladymyrov and Matthew Vowels.
</p>

This work is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

In this session we will look at the convolution operation and try to build some intuition about it.
Also we will look at one of the state-of-the art deep models, [Inception](https://arxiv.org/abs/1602.07261). It is designed to perform image recognition.

## 1. Load necessary libraries

In [ ]:
colab = True # set to True is using google colab

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" 

import sys
import shutil
import tarfile

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.utils import download_from_url

    
from scipy.ndimage import rotate

# We'll tell matplotlib to inline any drawn figures like so:


### Download libraries

In [ ]:
path = os.path.join(os.path.abspath('.')+'material.tgz')
url = 'https://github.com/neworldemancer/BMLWS/raw/main/tut_files/tpub0320.tgz'
# p = tf.keras.utils.get_file(path, url)
# Download compressed file with torch utils

p = download_from_url(url=url, path=path)

assert p==path
tar = tarfile.open(path, "r:gz")
tar.extractall()
tar.close()

## 2. Images

What is an image?

## 3. Convolutions

In fully connected network all inputs of a layer are connected to all neurons of the following layer:
<tr>
    <td> <img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/Perceptron.png" alt="drawing" width="30%"/></td> 
    <td> <img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/MLP.png" alt="drawing" width="50%"/></td> 
</tr> 
<br>In convolutional nets the same holds for each neighbourhood, and the weights are shared:<br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN1.png" alt="drawing" width="50%"/><br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN2.png" alt="drawing" width="50%"/><br>
<img src="https://github.com/neworldemancer/BMLWS/raw/main/figures/CNN3.png" alt="drawing" width="50%"/><br>


Let's see what a convolution is, and how it behaves.

### 1.1. Handkrafting filters

In [ ]:
#load image, convert to gray-scale and normalize
img_raw = plt.imread('ML3/chelsea.jpg')  # load RGB image (HWC)
img_raw = img_raw.mean(axis=2)  # convert to gray-scale by averaging over color channels
img_raw = img_raw[-256:, 100:356]  # crop to 256x256 pixels
img_raw = img_raw.astype(np.float32)  # convert to float32
img_raw = (img_raw-img_raw.mean())/img_raw.std()  # normalize to zero mean and unit variance

plt.imshow(img_raw, cmap='gray')  # show image
plt.grid(False)  # disable grid

In [ ]:
def conv_2d(x, flt):
    # x: B, C, H, W
    # flt: Co, Ci, Hf, Wf
    return F.conv2d(input=x, weight=flt, stride=1, padding=0, dilation=1)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

    @staticmethod
    def forward(x, flt):
        print(x.shape, flt.shape)
        y1 = conv_2d(x, flt)
        y2 = conv_2d(y1, flt)
        y3 = conv_2d(y2, flt)
        y4 = conv_2d(y3, flt)
        return x, y1, y2, y3, y4

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = Model().to(device)

In [ ]:
def get_convolved(img, flt):
    img_raw4d = img[np.newaxis,np.newaxis] # add batch (1) and channel (1) dimensions
    
    flt_mtx_np = np.array(flt, np.float32)
    flt_mtx_np = flt_mtx_np[np.newaxis, np.newaxis] # add N channels out (1) and N channels in (1) dimensions
    
    img_raw4d_t = torch.from_numpy(img_raw4d).to(device)
    flt_mtx_t = torch.from_numpy(flt_mtx_np).to(device) 
    
    res = model(img_raw4d_t, flt_mtx_t)
    res = [r.detach().cpu().numpy() for r in res]  # disable gradient tracking, move to cpu, convert to numpy
    res = [r[0,0] for r in res]  # remove batch and channel dimensions, only one channel in the output
    
    return res

Let's experiment with filter kernels:

In [ ]:
flt_mtx = [
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 1, 0, 0,],
    [ 0, 0, 0, 0, 0,],
    [ 0, 0, 0, 0, 0,],
] # identity transformation

ims_convolved = get_convolved(img_raw, flt_mtx)

n = len(ims_convolved)
fig, ax = plt.subplots(1, n+1, figsize=(n*4, 4))
for col in range(n):
    ax[col].imshow(ims_convolved[col], cmap='gray')  #, vmin=-3, vmax=3
    ax[col].grid(False)
    ax[col].set_title('conv %d'% col if col else 'raw')

ax[n].imshow(flt_mtx, cmap='gray')
ax[n].grid(False)
_=ax[n].set_title('filter')

1. experiment with different filters and understand what they do, e.g.:<br>
- identity transformation
- identity transformation with positive non-unit values
- identity transformation with negative unit value
- identity transformation off center
- blurring with box filter
- edge detection with + and - bands
- try whatever you like

2. experiment with convolution parameters: <br>
- padding = 1, 2, 3
- stride = 2
- dilation = 2

### 1.2. Most common filters

Here are most common filter kernels, in 1D:

|filter type| effect|
|-----|-----|
|gaussian| bluring|
|first derivative of gaussian|detection of edges|
|second derivative of gaussian|detection of peaks|


In [ ]:
def gaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y

def dgaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = - 2 * x * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y

def ddgaussian(n=5):
    x = np.linspace(-3, 3, n)
    y = - 2 * (2*x**2 - 1) * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi)
    return y
  
def ddgaussian2d(n=5):
    c = np.linspace(-3, 3, n)
    r = np.asarray([[np.sqrt(xi**2+yi**2) for xi in c] for yi in c])
    f = lambda x: (- 2 * (2*x**2 - 1) * np.exp(-x**2 * 0.5) / np.sqrt(2*np.pi))

    y = f(r)
    y -= y.mean()
    return y

In [ ]:
n = 30

gf = np.tile(gaussian(n)[np.newaxis], [n, 1])

dgf = np.tile(dgaussian(n)[np.newaxis], [n, 1])

ddgf = ddgaussian(n)
ddgf -= ddgf.mean()
ddgf = np.tile(ddgf[np.newaxis], [n, 1])

ddgf2d = ddgaussian2d(n)
rf2d = lambda:  np.random.normal(size=(5,5))


plt.plot(gf[0], label=r'$g(x)$')
plt.plot(dgf[0], label=r'$d g(x)/dx$')
plt.plot(ddgf[0], label=r'$d^2 g(x)/dx^2$')
plt.legend();

Gabor filter is a gaussian in one direction, and a derivative of gaussin on the other:

In [ ]:
gf.shape

In [ ]:
flt_mtx = gf*gf.transpose()

#flt_mtx = rotate(flt_mtx, 30, reshape=False)

plt.imshow(flt_mtx)
plt.grid(False)

In [ ]:
flt_mtx = gf*gf.transpose()    # gaussian filter
#flt_mtx = dgf*gf.transpose()   # gabor filter, 1st derivative of gaussian (edge detection)
#flt_mtx = ddgf*gf.transpose()  # 2nd derivative of gaussian (line detection)
#flt_mtx = ddgf2d               # 2nd derivative of gaussian, central symmetric, (spot detection)

#flt_mtx = rotate(flt_mtx, 45, reshape=False)

ims_convolved = get_convolved(img_raw, flt_mtx)

n = len(ims_convolved)
fig, ax = plt.subplots(1, n+1, figsize=(n*4, 4))
for col in range(n):
    ax[col].imshow(ims_convolved[col], cmap='gray')  #, vmin=-3, vmax=3
    ax[col].grid(False)
    ax[col].set_title('conv %d'% col if col else 'raw')

ax[n].imshow(flt_mtx, cmap='gray')
ax[n].grid(False)
_=ax[n].set_title('filter')

## 4. Exercise 20 min

Experiment with filters, try random, try to amplify some specific pattern eg whiskers or pupil, or perhaps make animation of filter effect depending on some parameter - e.g. size, angle, etc.

## 4. Homework

In last session we used fully connected network to clasify digits.
Try to build the convolutional network: use three convolutional layers, then flatten the ouput and apply 1 fully connected.
You can use the following helper function. Notice: there is a stride parameter. It allows to effectively downscale the feature maps.
To get an understanding of different convolution types, check the <a href="https://github.com/vdumoulin/conv_arithmetic">animations here</a>.

You can start with something like this:


In [ ]:
...


....

x = tf.keras.layers.Input(shape=(28, 28, 1))

l1 = tf.keras.layers.Conv2D(16, 3, name = 'C1', kernel_initializer='he_uniform', activation='relu')(x)
l2 = tf.keras.layers.Conv2D(32, 3, strides=2, name = 'C2', kernel_initializer='he_uniform', activation='relu')(l1)
l3 = tf.keras.layers.Conv2D(32, 3, strides=2, name = 'C3', kernel_initializer='he_uniform', activation='relu')(l2)

l3_f = tf.keras.layers.Flatten()(l3)

l4 = tf.keras.layers.Dense(units=32, name='l4', activation='relu')(l3_f)
l5 = tf.keras.layers.Dense(units=10, name='l5', activation='softmax')(l4)



In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.fc1 = nn.Linear(32*7*7, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # 28x28
        x = F.relu(self.conv2(x))  # 14x14
        x = F.relu(self.conv3(x))  # 7x7
        x = x.view(-1, 32*7*7)  # flatten 32x7x7 -> 32*7*7
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x)  # no activation, we will use cross-entropy loss with logits
        return x

model = Model().to(device)

In [ ]:
x_train = x_train_2d[:, np.newaxis]  # we need additional channel dimension, to get 4D (BHWC) dataset
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

....

Play with layer parameters. Can you get better performance than in fully connected network?

## 5. Load the model

Let's check some pretrained models

In [ ]:
#base_model = tf.keras.applications.InceptionV3(include_top=True) # ResNet50V2

# load a pretrained inception v5 model from torchhub
base_model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)

download_from_url(url='https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt', path='imagenet_classes.txt')

with open('imagenet_classes.txt') as f:
    class_names = [line.strip() for line in f.readlines()]

In [ ]:
import netron

In [ ]:
netron.start(r'C:\Users\newor/.cache\torch\hub\checkpoints\inception_v3_google-1a9a5a14.pth', )

## 6. Test the model

We will use one image to check model. `img_preproc` is croped to 299x299 pixels and slightly transformed to be used as imput for the model using `inception.prepare_training_img`. `inception.training_img_to_display` is then used to convert it to displayable one.


In [ ]:
sz = 299
img_raw = plt.imread('ML3/chelsea.jpg')

img_crop = img_raw.copy()[:sz, 100:100+sz]

_, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].imshow(img_raw)
axs[0].grid(False)
axs[1].imshow(img_crop)
axs[1].grid(False)
plt.show()

We need to apply same scaling to the input asa was done for training samples. This is done with a `preprocess_input` method corresponding to a model

In [ ]:
# ToDo: normalization: adapt with https://pytorch.org/hub/pytorch_vision_inception_v3/

In [ ]:
img_crop.transpose([2,0,1])[np.newaxis].shape

In [ ]:
probs = base_model(torch.from_numpy(img_raw.transpose([2,0,1])[np.newaxis]))

In [ ]:
probs = F.softmax(probs.logits, dim=1)[0]

In [ ]:
# get top 5 predictions
top5 = torch.topk(probs, 5)

In [ ]:
indices = top5.indices
probs = top5.values

In [ ]:
[class_names[i] for i in indices]

In [ ]:
all_preds = tf.keras.applications.inception_v3.decode_predictions(pred, top=-1)  #resnet_v2
plt.semilogy([prob for class_id, class_name, prob in all_preds[0]], '.');